# Аналіз дадзеных тэгу name у ОСМ Беларусі

## Зьмест

- Праблематыка
- Спампуем дамп ОСМ
- Усталёўваем залежанасьцьі
- Пошук сувязей дзеля падтрыманьня спасылачнай цэласнасьці

## Праблематыка

У беларускім ОСМ шырока выкарыстоўваюцца беларуская і расейская мова, для іх ёсьць адпаведнікі `name:be` і `name:ru`, таксама мовы выкарыстоўваюцца ў агульных тэгах як `name`, `addr:*` і іншых. Праблематка выкарыстоўваньня аднае, ці іншае, ці абедзьвух моваў апісанае тут https://wiki.openstreetmap.org/wiki/BE:Belarus_language_issues. Незалежна ад варыянту выкарыстоўваньня мовы павінны вытрымлівацца наступныя правілы: пошук на любое мове мусіць працаваць, павінна быць магчымасьць паказываць подпісы на любой мове (ці ў арыгінале, але гэтае правіла зараз не выконваецца), павінна захоўвацца спасылкавая цэласнасьць (што можа ўплываць на папярэднія два пункты).

Гэты аналіз ставіць мэтаю знайсьці адпаведныя катэгорыі і тэгі якія ўтрымліваюць кірылічныя значэньні тэгу name і падлічыць запаўняльнасьць тэгаў name:be, name:ru.

## Спампуем дамп ОСМ

In [27]:
!wget --backups=1 -N https://download.geofabrik.de/europe/belarus-latest.osm.pbf

--2022-04-04 07:31:51--  https://download.geofabrik.de/europe/belarus-latest.osm.pbf
Resolving download.geofabrik.de (download.geofabrik.de)... 95.216.28.113, 116.202.112.212
Connecting to download.geofabrik.de (download.geofabrik.de)|95.216.28.113|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 257087892 (245M) [application/octet-stream]
Saving to: ‘belarus-latest.osm.pbf’

belarus-latest.osm. 100%[===================>] 245.18M  2.51MB/s    in 95s     

2022-04-04 07:33:27 (2.57 MB/s) - ‘belarus-latest.osm.pbf’ saved [257087892/257087892]



## Загрузім дамп у postgis
- патрэбна толькі калі хочам атрымаць больш дакладныя дадзеныя, але можа не ўтрымліваць некаторыя дачыненьні

In [37]:
!PGPASSWORD=$POSTGRES_PASSWORD psql -h $POSTGRES_HOST -p $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB  -c "CREATE EXTENSION IF NOT EXISTS hstore"
!PGPASSWORD=$POSTGRES_PASSWORD psql -h $POSTGRES_HOST -p $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB  -c "DROP MATERIALIZED VIEW IF EXISTS planet_osm_data"
!PGPASSWORD=$POSTGRES_PASSWORD psql -h $POSTGRES_HOST -p $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB  -c "DROP MATERIALIZED VIEW IF EXISTS planet_osm_region"

NOTICE:  extension "hstore" already exists, skipping
CREATE EXTENSION
NOTICE:  materialized view "planet_osm_data" does not exist, skipping
DROP MATERIALIZED VIEW
NOTICE:  materialized view "planet_osm_region" does not exist, skipping
DROP MATERIALIZED VIEW


In [38]:
!PGPASSWORD=$POSTGRES_PASSWORD osm2pgsql -H $POSTGRES_HOST -P $POSTGRES_POST -U $POSTGRES_USER -d $POSTGRES_DB -v -l -j -G -x --hstore-add-index -C $OSM2PGSQL_CACHE -S /usr/share/osm2pgsql/default.style belarus-latest.osm.pbf

2022-04-04 07:42:41  osm2pgsql version 1.6.0
2022-04-04 07:42:41  [0] Database version: 14.2
2022-04-04 07:42:41  [0] PostGIS version: 3.2
2022-04-04 07:42:41  [0] Reading file: belarus-latest.osm.pbf
2022-04-04 07:42:41  [0] Started pool with 4 threads.
2022-04-04 07:42:41  [0] Using projection SRS 4326 (Latlong)
2022-04-04 07:42:41  [0] Using built-in tag transformations
2022-04-04 07:42:41  [0] Middle 'ram' options:
2022-04-04 07:42:41  [0]   locations: true
2022-04-04 07:42:41  [0]   way_nodes: true
2022-04-04 07:42:41  [0]   nodes: false
2022-04-04 07:42:41  [0]   untagged_nodes: true
2022-04-04 07:42:41  [0]   ways: false
2022-04-04 07:42:41  [0]   relations: false
2022-04-04 07:42:41  [0] Setting up table 'planet_osm_point'
2022-04-04 07:42:41  [0] Setting up table 'planet_osm_line'
2022-04-04 07:42:41  [0] Setting up table 'planet_osm_polygon'
2022-04-04 07:42:42  [0] Setting up table 'planet_osm_roads'
2022-04-04 07:44:07  [0] Reading input files done in 85s (1m 25s).         

## Усталюем залежнасьці

In [2]:
!pip install pandas matplotlib psycopg2-binary https://github.com/lechup/imposm-parser/archive/python3.zip

  Using cached https://github.com/lechup/imposm-parser/archive/python3.zip
  Preparing metadata (setup.py) ... done


## Вызначым катэгорыі

In [29]:
import os
import re
from collections import defaultdict, Counter

from imposm.parser import OSMParser
import psycopg2

import pandas as pd
pd.set_option('display.max_rows', None)


cirylic_chars = frozenset('абвгдеёжзіийклмнопрстуўфхцчшщьыъэюяАБВГДЕЁЖЗІИІЙКЛМНОПРСТУФХЦЧШЩЬЫЪЭЮЯ')


In [49]:
categories_rules = {
    'admin': [
#         ['boundary', 'administrative'],
        ['admin_level', '2'],
        ['admin_level', '4'],
        ['admin_level', '6'],
        ['admin_level', '8'],
        ['admin_level', '9'],
    ],
    'place': [
        ['place', 'city'],
        ['place', 'town'],
        ['place', 'village'],
        ['place', 'hamlet'],
        ['place', 'isolated_dwelling'],
        ['admin_level', None],
    ],
    'locality': [
        ['place', 'allotments'],
        ['place', 'locality'],
        ['abandoned:place', None],
    ],
    'suburb': [
        ['landuse', 'commercial'],
        ['landuse', 'construction'],
        ['landuse', 'education'],
        ['landuse', 'industrial'],
        ['landuse', 'residential'],
        ['landuse', 'retail'],
        ['landuse', 'allotments'],
        ['place', None],
        ['residential', None],
        ['industrial', None],
    ],
    'highway': [
        ['highway', 'motorway'],
        ['highway', 'trunk'],
        ['highway', 'primary'],
        ['highway', 'secondary'],
        ['highway', 'tertiary'],
        ['highway', 'unclassified'],
        ['highway', 'residential'],
        ['highway', 'service'],
        ['highway', 'track'],
        ['highway', None],
        ['type', 'associatedStreet'],
    ],
    'public_transport': [
        ['highway', 'bus_stop'],
        ['public_transport', None],
        ['route', None],
        ['type', 'route'],
        ['railway', None],
        ['route_master', None],
    ],
    'infrastructure': [
        ['tunnel', None],
        ['barrier', None],
        ['power', None],
        ['bridge', None],
        ['substation', None],
        ['emergency', None],
        ['ele', None],
        ['man_made', None],
        ['embankment', None],
    ],
    'amenity': [
        ['amenity', 'place_of_worship'],
        ['amenity', 'school'],
        ['amenity', 'kindergarten'],
#         ['amenity', 'cafe'],
#         ['amenity', 'atm'],
#         ['amenity', 'pharmacy'],
#         ['amenity', 'bank'],
#         ['amenity', 'post_office'],
#         ['amenity', 'fast_food'],
#         ['amenity', 'fuel'],
#         ['amenity', 'community_centre'],
#         ['amenity', 'hospital'],
#         ['amenity', 'police'],
#         ['amenity', 'restaurant'],
#         ['amenity', 'clinic'],
#         ['amenity', 'doctors'],
#         ['amenity', 'library'],
#         ['amenity', 'bar'],
        ['amenity', None],
        ['shop', None],
        ['leisure', None],
        ['sport', None],
        ['craft', 'shoemaker'],
        ['clothes', None],
    ],
    'government': [
        ['healthcare', None],
        ['office', 'government'],
        ['government', None],
        ['military', None],
    ],
    'office': [
        ['office', None],
    ],
    'building': [
        ['building', 'industrial'],
        ['building', 'service'],
        ['building', 'retail'],
        ['building', 'school'],
        ['building', 'kindergarten'],
        ['building', 'commercial'],
        ['building', 'church'],
        ['building', 'warehouse'],
        ['building', 'public'],
        ['building', 'dormitory'],
        ['building', 'hospital'],
        ['building', 'warehouse'],
        ['building', None],
    ],
    'tourism': [
        ['tourism', None],
        ['historic', None],
        ['memorial', None],
        ['ruins', None],
        ['information', None],
        ['attraction', None],
        ['resort', None],
        ['artwork_type', None],
    ],
    'water': [
        ['waterway', 'drain'],
        ['waterway', 'ditch'],
        ['waterway', 'stream'],
        ['waterway', 'river'],
        ['waterway', 'canal'],
        ['waterway', None],
        ['type', 'waterway'],
        ['water', None],
        ['natural', 'water'],
        ['natural', 'spring'],
    ],
    'natural': [
        ['natural', None],
        ['place', 'island'],
        ['place', 'islet'],
        ['landuse', None],
    ],
}

usage = defaultdict(set)
categories_rules2 = {}
for category, group in categories_rules.items():
    if category not in categories_rules2:
        categories_rules2[category] = []
    for tag, value in group:
        if value is not None:
            categories_rules2[category].append([tag, True, {value}])
            usage[tag].add(value)                          
for category, group in categories_rules.items():
    if category not in categories_rules2:
        categories_rules2[category] = []
    for tag, value in group:
        if value is None:
            categories_rules2[category].append([tag, False, usage[tag]])

## Падлічам статыстыку для дампу
- дамп падліча ўсе дадзеныя, але можа быць трошкі недакладным таму што не ўлічвае грубую абрэзку Беларусі

In [50]:
key_counter = defaultdict(lambda: defaultdict(list))

categories_tags = {}
categories_rules_tags_set = {}
for category, group in categories_rules2.items():
    for tag, eq, values in group:
        if tag not in categories_tags:
            categories_tags[tag] = {category}
        else:
            categories_tags[tag].add(category)

            
def process(params):
    for _, tags, _ in params:
        if 'name' not in tags:
            continue
        if not (frozenset(tags['name']) & cirylic_chars):
            continue
        categories = {category for tag in categories_tags.keys() & tags.keys() for category in categories_tags[tag]}
        for tag in ['name', 'name:be', 'name:ru']:
            if tag not in tags:
                continue
            value = tags[tag]
            cyr = frozenset(value) & cirylic_chars
            if not cyr:
                continue
            match = False
            for category in categories:
                group = categories_rules2[category]
                category_match = False
                for i, (k, eq, vv) in enumerate(group):
                    if k not in tags:
                        continue
                    if eq:
                        if tags[k] in vv:
                            if not category_match:
                                key_counter[(category,)][tag].append(value)
                                match = category_match = True
                            key_counter[(category, i)][tag].append(value)   
                    else:
                        if tags[k] not in vv:
                            if not category_match:
                                key_counter[(category,)][tag].append(value)
                                match = category_match = True
                            key_counter[(category, i)][tag].append(value)   
            if not match:
                key_counter[('other',)][tag].append(value)
                

OSMParser(
    nodes_callback=process,
    ways_callback=process,
    relations_callback=process,
).parse('belarus-latest.osm.pbf')


In [51]:
data = []
for c in list(categories_rules) + ['other']:
    name_cnt = len(key_counter[(c,)]['name'])
    name_uniq = len(set(key_counter[(c,)]['name']))
    name_be_cnt = len(key_counter[(c,)]['name:be'])
    name_be_uniq = len(set(key_counter[(c,)]['name:be']))
    name_ru_cnt = len(key_counter[(c,)]['name:ru'])
    name_ru_uniq = len(set(key_counter[(c,)]['name:ru']))
    data.append([
        '#', c, 
        name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
        name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
    ])
    if c == 'other':
        continue
    for i, (k, eq, vv) in enumerate(categories_rules2[c]):
        if eq:
            tag = f'{k} = {list(vv)[0]}'
        else:
            tag = f'{k} = *'
        name_cnt = len(key_counter[(c, i)]['name'])
        name_uniq = len(set(key_counter[(c, i)]['name']))
        name_be_cnt = len(key_counter[(c, i)]['name:be'])
        name_be_uniq = len(set(key_counter[(c, i)]['name:be']))
        name_ru_cnt = len(key_counter[(c, i)]['name:ru'])
        name_ru_uniq = len(set(key_counter[(c, i)]['name:ru']))
        data.append([
            '', tag, 
            name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
            name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
        ])


df = pd.DataFrame(data, columns=[
    'lvl', 'category', 
    'all name', 'all name:be', 'all name:ru', 'all name:be%', 'all name:ru%',
    'uniq name', 'uniq name:be', 'uniq name:ru', 'uniq name:be%', 'uniq name:ru%',
])
df.to_csv('dump.csv')
df.style.set_properties(**{'text-align': 'left'}).background_gradient('YlOrRd', subset=[
    'all name:be%', 'all name:ru%', 'uniq name:be%', 'uniq name:ru%',
]).apply(lambda row: [("font-weight: bold" if row.loc['lvl'] == '#' else '') for _ in row], axis=1)

,lvl,category,all name,all name:be,all name:ru,all name:be%,all name:ru%,uniq name,uniq name:be,uniq name:ru,uniq name:be%,uniq name:ru%
0,#,admin,2388,2255,2098,0.944305,0.878559,1571,1479,1481,0.941439,0.942712
1,,admin_level = 2,702,638,477,0.908832,0.679487,109,83,85,0.761468,0.779817
2,,admin_level = 4,73,72,72,0.986301,0.986301,40,38,38,0.950000,0.950000
3,,admin_level = 6,234,213,216,0.910256,0.923077,189,169,173,0.894180,0.915344
4,,admin_level = 8,1352,1306,1307,0.965976,0.966716,1234,1195,1192,0.968395,0.965964
5,,admin_level = 9,27,26,26,0.962963,0.962963,14,13,13,0.928571,0.928571
6,#,place,45627,45139,45341,0.989305,0.993732,15739,15668,15594,0.995489,0.990787
7,,place = city,31,31,31,1.000000,1.000000,16,16,16,1.000000,1.000000
8,,place = town,274,273,274,0.996350,1.000000,138,138,138,1.000000,1.000000
9,,place = village,5031,4987,5015,0.991254,0.996820,2227,2208,2215,0.991468,0.994612


## Падлічам статыстыку для выгрузкі ў postgis
- вынік будзе больш дакладным, але можа ня ўлічываць дачыненьні што не пераносяцца ў postgis

In [52]:
query_template = """
SELECT '{category}' AS category, {num} AS num, g.tags->'name' AS name, g.tags->'name:be' AS name_be, g.tags->'name:ru' AS name_ru
FROM {table} g
WHERE {condition}
-- ({cyr})
"""
# tables = ['planet_osm_line', 'planet_osm_point', 'planet_osm_polygon']
tables = ['planet_osm_data']
cyr_regexp = '|'.join(cirylic_chars)

queries = []
exclude = []
for category, group in categories_rules2.items():
    conditions = []
    for i, (k, eq, vv) in enumerate(group):
        if vv:
            eq_str = 'IN' if eq else 'NOT IN'
            vv_str = ','.join(f"'{v}'" for v in vv)
            condition = f"g.tags->'{k}' {eq_str} ({vv_str})"
        elif not eq:
            condition = f"g.tags->'{k}' IS NOT NULL"
        else:
            raise ValueError()
        conditions.append(condition)
        exclude.append(condition)
        for table in tables:
            query = query_template.format(category=category, num=i, table=table, cyr=cyr, condition=condition)
            queries.append(query)
    condition = ' OR '.join(f'({c})' for c in conditions)
    for table in tables:
        query = query_template.format(category=category, num=-1, table=table, cyr=cyr, condition=condition)
        queries.append(query)
condition = ' OR '.join(f'({c})' for c in exclude)
for table in tables:
    query = query_template.format(category='other', num=-1, table=table, cyr=cyr, condition=f'NOT ({condition})')
    queries.append(query)
query = ' UNION ALL '.join(queries)

print(len(queries))


114


In [53]:
key_counter = defaultdict(lambda: defaultdict(list))

REGION_VIEW_SQL = """
CREATE MATERIALIZED VIEW IF NOT EXISTS planet_osm_region AS

SELECT ST_Buffer(way, -0.000000001) AS way
FROM planet_osm_polygon
WHERE osm_id = -59065
LIMIT 1
"""
REGION_WAY_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_region_way_idx" ON planet_osm_region USING GIST (way)
"""
REGION_ANALYZE_SQL = "ANALYZE planet_osm_region"

OSM_DATA_VIEW_SQL = f"""
CREATE MATERIALIZED VIEW IF NOT EXISTS planet_osm_data AS

SELECT
    g.osm_id AS osm_id,
    'node' AS osm_type,
    'point' AS kind,
    g.tags AS tags
FROM planet_osm_point g
INNER JOIN planet_osm_region p
ON ST_Intersects(p.way, g.way)
WHERE g.tags->'name' ~ '({cyr_regexp})'

UNION ALL

SELECT
    ABS(g.osm_id) AS osm_id,
    CASE WHEN g.osm_id < 0 THEN 'relation' ELSE 'way' END AS osm_type,
    'line' AS kind,
    g.tags AS tags
FROM planet_osm_line g
INNER JOIN planet_osm_region p
ON ST_Intersects(p.way, g.way)
WHERE g.tags->'name' ~ '({cyr_regexp})'

UNION ALL

SELECT
    ABS(g.osm_id) AS osm_id,
    CASE WHEN g.osm_id < 0 THEN 'relation' ELSE 'way' END AS osm_type,
    'poly' AS kind,
    g.tags AS tags
FROM planet_osm_polygon g
INNER JOIN planet_osm_region p
ON ST_Intersects(p.way, g.way)
WHERE g.tags->'name' ~ '({cyr_regexp})'
"""
OSM_DATA_OSM_ID_TYPE_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_data_osm_id_type_idx" ON planet_osm_data (osm_id, osm_type)
"""
OSM_DATA_TAGS_INDEX_SQL = """
CREATE INDEX IF NOT EXISTS "planet_osm_data_tags_idx" ON planet_osm_data USING GIN (tags)
"""
OSM_DATA_ANALYZE_SQL = "ANALYZE planet_osm_data"


conn = psycopg2.connect(
    host=os.environ['POSTGRES_HOST'],
    dbname=os.environ['POSTGRES_DB'],
    user=os.environ['POSTGRES_USER'],
    password=os.environ['POSTGRES_PASSWORD'],
)
cur = conn.cursor()

cur.execute(REGION_VIEW_SQL)
cur.execute(REGION_WAY_INDEX_SQL)
cur.execute(REGION_ANALYZE_SQL)

cur.execute(OSM_DATA_VIEW_SQL)
cur.execute(OSM_DATA_OSM_ID_TYPE_INDEX_SQL)
cur.execute(OSM_DATA_TAGS_INDEX_SQL)
cur.execute(OSM_DATA_ANALYZE_SQL)

for i, query in enumerate(queries, 1):
    cur.execute(query)
    records = cur.fetchall()
    for category, num, name, name_be, name_ru in records:
        key = (category,) if num == -1 else (category, num)
        key_counter[key]['name'].append(name)
        if name_be is not None:
            key_counter[key]['name:be'].append(name_be)
        if name_ru is not None:
            key_counter[key]['name:ru'].append(name_ru)
cur.close()
conn.close()

In [54]:
data = []
for c in list(categories_rules) + ['other']:
    name_cnt = len(key_counter[(c,)]['name'])
    name_uniq = len(set(key_counter[(c,)]['name']))
    name_be_cnt = len(key_counter[(c,)]['name:be'])
    name_be_uniq = len(set(key_counter[(c,)]['name:be']))
    name_ru_cnt = len(key_counter[(c,)]['name:ru'])
    name_ru_uniq = len(set(key_counter[(c,)]['name:ru']))
    data.append([
        '#', c, 
        name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
        name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
    ])
    if c == 'other':
        continue
    for i, (k, eq, vv) in enumerate(categories_rules2[c]):
        if eq:
            tag = f'{k} = {list(vv)[0]}'
        else:
            tag = f'{k} = *'
        name_cnt = len(key_counter[(c, i)]['name'])
        name_uniq = len(set(key_counter[(c, i)]['name']))
        name_be_cnt = len(key_counter[(c, i)]['name:be'])
        name_be_uniq = len(set(key_counter[(c, i)]['name:be']))
        name_ru_cnt = len(key_counter[(c, i)]['name:ru'])
        name_ru_uniq = len(set(key_counter[(c, i)]['name:ru']))
        data.append([
            '', tag, 
            name_cnt, name_be_cnt, name_ru_cnt, name_be_cnt/(name_cnt or 1), name_ru_cnt/(name_cnt or 1),
            name_uniq, name_be_uniq, name_ru_uniq, name_be_uniq/(name_uniq or 1), name_ru_uniq/(name_uniq or 1),
        ])


df = pd.DataFrame(data, columns=[
    'lvl', 'category', 
    'all name', 'all name:be', 'all name:ru', 'all name:be%', 'all name:ru%',
    'uniq name', 'uniq name:be', 'uniq name:ru', 'uniq name:be%', 'uniq name:ru%',
])
df.to_csv('postgis.csv')
df.style.set_properties(**{'text-align': 'left'}).background_gradient('YlOrRd', subset=[
    'all name:be%', 'all name:ru%', 'uniq name:be%', 'uniq name:ru%',
]).apply(lambda row: [("font-weight: bold" if row.loc['lvl'] == '#' else '') for _ in row], axis=1)

,lvl,category,all name,all name:be,all name:ru,all name:be%,all name:ru%,uniq name,uniq name:be,uniq name:ru,uniq name:be%,uniq name:ru%
0,#,admin,4164,4161,4157,0.999280,0.998319,1393,1395,1387,1.001436,0.995693
1,,admin_level = 2,4,4,4,1.000000,1.000000,3,3,3,1.000000,1.000000
2,,admin_level = 4,165,164,164,0.993939,0.993939,32,30,30,0.937500,0.937500
3,,admin_level = 6,716,714,710,0.997207,0.991620,165,164,161,0.993939,0.975758
4,,admin_level = 8,3214,3214,3214,1.000000,1.000000,1191,1195,1191,1.003359,1.000000
5,,admin_level = 9,65,65,65,1.000000,1.000000,13,13,13,1.000000,1.000000
6,#,place,75360,74975,75300,0.994891,0.999204,15569,15640,15547,1.004560,0.998587
7,,place = city,75,75,75,1.000000,1.000000,16,16,16,1.000000,1.000000
8,,place = town,474,474,474,1.000000,1.000000,137,138,137,1.007299,1.000000
9,,place = village,8584,8574,8582,0.998835,0.999767,2196,2206,2196,1.004554,1.000000
